In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jul 11 18:06:00 2022

@author: Beau.Uriona
"""

import folium
import pandas as pd
import geopandas as gpd
from requests import Session

# Import the function to get connect to the db
from snowexsql.db import get_db
from snowexsql.conversions import query_to_geopandas
from snowexsql.data import SiteData, PointData, LayerData, ImageData

# Get the snow ex sites
SNOWEX_DB_NAME = 'snow:hackweek@db.snowexdata.org/snowex'

engine, session = get_db(SNOWEX_DB_NAME)
qry = session.query(SiteData.site_name, SiteData.site_id, SiteData.geom)

gdf_snow_ex = query_to_geopandas(qry, engine).drop_duplicates(subset=["site_name"])
#gdf_snow_ex.rename(columns={"geom": "geometry"}, inplace=True)

print(f"The CRS of the Snow Ex metadata is - {gdf_snow_ex.crs}")
gdf_snow_ex_buffer = gdf_snow_ex.buffer(50000)
snow_ex_bounds = gdf_snow_ex.to_crs(4326).total_bounds
print(f"The bounding box of the snoe Ex sites is - {snow_ex_bounds}")
snow_ex_map = gdf_snow_ex.explore(tooltip=[i for i in gdf_snow_ex.columns if not i.lower().startswith("geo")])
snow_ex_map = gdf_snow_ex_buffer.explore(m=snow_ex_map)

# Get SNOTEL Sites
AWDB_API_DOMAIN = "https://api.snowdata.info"
sntl_meta_url = f"{AWDB_API_DOMAIN}/stations/getMeta?network=SNTL&format=geojson"
# Get the data and rename everything to match the snow ex columns
gdf_sntl = gpd.read_file(sntl_meta_url).rename(
    columns={"name": "site_name", "stationTriplet": "site_id"}
)
print(f"The CRS of the Snotel metadata is - {gdf_sntl.crs}, better change it")
gdf_sntl = gdf_sntl.to_crs(gdf_snow_ex.crs)
print(f"The CRS of the Snotel metadata is now - {gdf_sntl.crs}, all good!")
# Only keep the columns we care about... for now
gdf_sntl = gdf_sntl.drop(
    columns=[i for i in gdf_sntl.columns if i not in ["geometry", "site_id", "site_name"]]
)
gdf_sntl_buffer = gdf_sntl.buffer(50000)
sntl_bounds = gdf_sntl.to_crs(4326).total_bounds
print(f"The bounding box of the Snotel sites is - {snow_ex_bounds}")
sntl_map = gdf_sntl.explore(tooltip=[i for i in gdf_snow_ex.columns if not i.lower().startswith("geo")])
sntl_map = gdf_sntl_buffer.explore(m=sntl_map)

/srv/conda/envs/notebook/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.11.0-CAPI-1.17.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


The CRS of the Snow Ex metadata is - epsg:26912
The bounding box of the snoe Ex sites is - [-113.72623049   35.88858147 -108.0418733    44.30447084]
The CRS of the Snotel metadata is - epsg:4979, better change it
The CRS of the Snotel metadata is now - epsg:26912, all good!
The bounding box of the Snotel sites is - [-113.72623049   35.88858147 -108.0418733    44.30447084]


In [2]:
snow_ex_map

In [3]:
gdf_nearest = gpd.sjoin_nearest(gdf_snow_ex.to_crs(26912), gdf_sntl.to_crs(26912), distance_col="distance").set_geometry(col="geom")
gdf_nearest

,site_name_left,site_id_left,geom,index_right,site_name_right,site_id_right,distance
0,Grand Mesa,2C12,POINT (742229.000 4326414.000),550,Mesa Lakes,622:CO:SNTL,12408.883217
158,American River Basin,Caples Lake,POINT (757215.939 4288786.947),550,Mesa Lakes,622:CO:SNTL,38566.781003
574,Sagehen Creek,Forest,POINT (737377.328 4368090.443),550,Mesa Lakes,622:CO:SNTL,44312.463166
68,Boise River Basin,Banner Open,POINT (640697.679 4907203.549),544,Marquette,616:WY:SNTL,458.727311
114,East River,Open 6,POINT (328364.133 4310498.494),233,Rockwood GS,1309:UT:SNTL,63147.714178
210,Cameron Pass,Joe Wright,POINT (424323.590 4486303.233),688,Snowbird,766:UT:SNTL,20512.529383
336,Fraser Experimental Forest,JPL 1,POINT (424541.156 4417657.215),219,Santaquin Meadows,1280:UT:SNTL,14183.045596
462,Jemez River,BA Flux Tower,POINT (361743.373 3972674.013),115,Chalender,1139:AZ:SNTL,81461.125576
478,Little Cottonwood Canyon,Alta Collins,POINT (446677.235 4491448.536),232,Atwater,1308:UT:SNTL,2223.911763
492,Mammoth Lakes,CUES,POINT (320983.987 4168172.701),195,Suu Ranch,1248:UT:SNTL,10175.063048


In [4]:
nearest_map = gdf_nearest.explore(
    "distance", 
    tooltip=[i for i in gdf_nearest.columns if not i.lower().startswith("geo")],
    marker_kwds={"radius": 10},
    vmin=0,
)
nearest_map

In [5]:

import os
from os import path, makedirs

curr_path = os.getcwd()
output_dir = path.join(curr_path, "output")
makedirs(output_dir, exist_ok=True)
shp_dir = path.join(output_dir, "shp")
makedirs(shp_dir, exist_ok=True)
geojson_dir = path.join(output_dir, "geojson")
makedirs(geojson_dir, exist_ok=True)

snow_ex_dir = path.join(shp_dir, "snow-ex")
makedirs(snow_ex_dir, exist_ok=True)
gdf_snow_ex.to_crs(4326, inplace=True)
gdf_snow_ex.set_geometry(col="geom", inplace=True)
gdf_snow_ex.to_file(path.join(snow_ex_dir, "snow_ex_stations.shp"), crs=4326)
gdf_snow_ex.to_file(path.join(geojson_dir, "snow_ex_stations.json"), crs=4326, driver='GeoJSON')
                    
sntl_dir = path.join(shp_dir, "sntl")
makedirs(sntl_dir, exist_ok=True)
gdf_sntl.to_crs(4326, inplace=True)
gdf_sntl.to_file(path.join(sntl_dir, "sntl_stations.shp"), crs=4326)
gdf_sntl.to_file(path.join(geojson_dir, "sntl_stations.json"), crs=4326, driver='GeoJSON')
                 
nearest_dir = path.join(shp_dir, "nearest")
makedirs(nearest_dir, exist_ok=True)
gdf_nearest.to_file(path.join(nearest_dir,  "nearest_stations.shp"), crs=4326)
gdf_nearest.to_file(path.join(geojson_dir,  "nearest_stations.json"), crs=4326, driver='GeoJSON')

/tmp/ipykernel_726/1912461499.py:27: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_nearest.to_file(path.join(nearest_dir,  "nearest_stations.shp"), crs=4326)
